In [1]:
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 MB 24.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 KB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 93.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 91.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.2/461.2 KB 49.3 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-transform 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
grpcio-status 1.44.0 requires grpcio>=1.44.0, but you have grpcio 1.43.0 which is incompatible.
apache-beam 2.37.0 requires dill<0.3.2,>=0.3.

In [2]:
import ray

In [3]:
!which -a ray

/opt/conda/bin/ray


In [4]:
!conda env list

# conda environments:
#
base                  *  /opt/conda



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
import keras
import tempfile

In [2]:

import numpy as np
import tensorflow as tf


def mnist_dataset(batch_size):
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    # The `x` arrays are in uint8 and have values in the [0, 255] range.
    # You need to convert them to float32 with values in the [0, 1] range.
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (x_train, y_train)).batch(batch_size)
    print("HI=========================================================================================================================")
    print(train_dataset.cardinality())
    return train_dataset


def build_and_compile_cnn_model():
    
    model = Sequential()

    model.add(Conv2D(32, 3, activation='relu', padding='same', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(Conv2D(128, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(Conv2D(256, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())

    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 1)) # default stride is 2
    model.add(BatchNormalization())

    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(Conv2D(512, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 1)) # default stride is 2
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [3]:

import json
import os

def train_func_distributed():
    per_worker_batch_size = 64
    # This environment variable will be set by Ray Train.
    tf_config = json.loads(os.environ['TF_CONFIG'])
    num_workers = len(tf_config['cluster']['worker'])

    strategy = tf.distribute.MultiWorkerMirroredStrategy()

    global_batch_size = per_worker_batch_size * num_workers
    multi_worker_dataset = mnist_dataset(global_batch_size)

    with strategy.scope():
        # Model building/compiling need to be within `strategy.scope()`.
        multi_worker_model = build_and_compile_cnn_model()

    multi_worker_model.fit(multi_worker_dataset, epochs=3)


In [ ]:
from ray.train import Trainer

trainer = Trainer(backend="tensorflow", num_workers=1, use_gpu=True)

# For GPU Training, set `use_gpu` to True.
# trainer = Trainer(backend="tensorflow", num_workers=4, use_gpu=True)

trainer.start()
results = trainer.run(train_func_distributed)
trainer.shutdown()


2022-04-18 07:06:18,227	INFO trainer.py:223 -- Trainer logs will be logged in: /home/jupyter/ray_results/train_2022-04-18_07-06-18
2022-04-18 07:06:23,298	INFO trainer.py:229 -- Run results will be logged in: /home/jupyter/ray_results/train_2022-04-18_07-06-18/run_001
(BaseWorkerMixin pid=10036) 2022-04-18 07:06:23.339154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
(BaseWorkerMixin pid=10036) 2022-04-18 07:06:23.349492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
(BaseWorkerMixin pid=10036) 2022-04-18 07:06:23.350144: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NU

(BaseWorkerMixin pid=10036) HI=========================================================================================================================
(BaseWorkerMixin pid=10036) tf.Tensor(938, shape=(), dtype=int64)


(BaseWorkerMixin pid=10036) 2022-04-18 07:06:25.092721: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
(BaseWorkerMixin pid=10036) op: "TensorSliceDataset"
(BaseWorkerMixin pid=10036) input: "Placeholder/_0"
(BaseWorkerMixin pid=10036) input: "Placeholder/_1"
(BaseWorkerMixin pid=10036) attr {
(BaseWorkerMixin pid=10036)   key: "Toutput_types"
(BaseWorkerMixin pid=10036)   value {
(BaseWorkerMixin pid=10036)     list {
(BaseWorkerMixin pid=10036)       type: DT_FLOAT
(BaseWorkerMixin pid=10036)       type: DT_INT64
(BaseWorkerMixin pid=10036)     }
(BaseWorkerMixin pid=10036)   }
(BaseWorkerMixin pid=10036) }
(BaseWorkerMixin pid=10036) attr {
(BaseWorkerMixin pid=10036)   key: "_cardinality"
(BaseWorkerMixin pid=10036)   value {
(BaseWorkerMixin pid=10036)     i: 60000
(BaseWor

(BaseWorkerMixin pid=10036) Epoch 1/15


(BaseWorkerMixin pid=10036) /opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
(BaseWorkerMixin pid=10036)   return dispatch_target(*args, **kwargs)
(BaseWorkerMixin pid=10036) 2022-04-18 07:06:29.711425: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


938/938 [==============================] - 45s 40ms/step - loss: 0.2990 - accuracy: 0.9223
(BaseWorkerMixin pid=10036) Epoch 2/15
938/938 [==============================] - 38s 40ms/step - loss: 0.1722 - accuracy: 0.9564
(BaseWorkerMixin pid=10036) Epoch 3/15
938/938 [==============================] - 38s 40ms/step - loss: 0.0839 - accuracy: 0.9799
(BaseWorkerMixin pid=10036) Epoch 4/15
938/938 [==============================] - 38s 40ms/step - loss: 0.0747 - accuracy: 0.9823
(BaseWorkerMixin pid=10036) Epoch 5/15
807/938 [========================>.....] - ETA: 5s - loss: 0.1078 - accuracy: 0.9765
